In [2]:
import pandas as pd

In [3]:
titanic_data = pd.read_csv('Data/titanic-train.csv.txt')

In [4]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
titanic_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Splitting feature data & target data
* Dropping less impotant columns

In [6]:
feature_data = titanic_data.drop(['Survived'], axis=1)

In [7]:
target_data = titanic_data.Survived

In [8]:
feature_data.drop(['Name','Cabin','PassengerId'], axis=1, inplace=True)

In [9]:
feature_data.drop(['Ticket'],axis=1,inplace=True)

In [10]:
feature_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [11]:
feature_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [12]:
feature_data.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

* Seperating features based on types

In [13]:
cat_cols = list(feature_data.select_dtypes(include=['object']))
num_cols = list(feature_data.select_dtypes(exclude=['object']))

* Realizing missing values, create imputers

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
si_cat_cols = SimpleImputer(strategy='constant', fill_value='S')
si_num_cols = SimpleImputer(strategy='median')

In [16]:
num_data = si_num_cols.fit_transform(feature_data[num_cols])

In [17]:
cat_data = si_cat_cols.fit_transform(feature_data[cat_cols])

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
ohe = OneHotEncoder()

In [28]:
cat_data_ohe = ohe.fit_transform(cat_data).toarray()
cat_data_ohe

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

##### Combine transformed data

In [21]:
import numpy as np
feature_data_tf = np.hstack([num_data, cat_data_ohe])

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
trainx, testx, trainy, testy = train_test_split(feature_data_tf, target_data)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [25]:
models = [ LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier()]
trained_models = []

for model in models:
    model.fit(trainx,trainy)
    trained_models.append(model)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
for model in trained_models:
    print (model.score(testx,testy))

0.7937219730941704
0.7533632286995515
0.8071748878923767
